In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


In [2]:
# output_data_file

output_data_file = "Output_Data/weather.csv"


types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,hobart,75,AU,1603497192,54,-42.88,147.33,60.01,9.17
1,anadyr,75,RU,1603497356,100,64.75,177.48,23.00,4.47
2,bluff,95,NZ,1603497356,51,-46.60,168.33,71.01,15.01
3,tiznit,62,MA,1603497356,69,29.58,-9.50,62.10,0.87
4,cape town,2,CL,1603497356,44,-36.41,-71.90,60.80,11.41
...,...,...,...,...,...,...,...,...,...
520,podosinovets,100,RU,1603497436,97,60.28,47.07,37.65,7.25
521,yar-sale,90,RU,1603497436,95,66.83,70.83,20.97,8.72
522,shubarkuduk,100,KZ,1603497437,76,49.14,56.49,38.37,18.14
523,sandpoint,90,US,1603497437,92,48.28,-116.55,33.01,5.82


In [3]:
#Humidity Heatmap

#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

In [4]:
gmaps.configure(g_key)

In [5]:
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df["Humidity"].astype(float)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
 #Create new DataFrame fitting weather criteria

#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.

In [8]:
narrow_df = pd.DataFrame(weather_data_df, columns = ["City", "Country","Humidity","Max Temp", "Wind Speed", "Cloudiness","Lat","Lng"])

max_temp = (narrow_df["Max Temp"] <= 80) & (types_df["Max Temp"] > 70)
wind_speed = narrow_df["Wind Speed"] < 10
cloudiness = narrow_df["Cloudiness"] == 0
narrow_df[max_temp & wind_speed & cloudiness]

,City,Country,Humidity,Max Temp,Wind Speed,Cloudiness,Lat,Lng
107,hambantota,LK,94,77.00,4.70,0,6.12,81.12
210,rajnagar,IN,41,70.32,2.84,0,24.88,79.92
231,roebourne,AU,44,75.92,9.60,0,-20.78,117.13
246,cayenne,GF,94,78.80,9.71,0,4.93,-52.33
249,bhikangaon,IN,58,72.46,1.57,0,21.87,75.95
281,talaja,IN,48,78.94,2.24,0,21.35,72.05
308,francistown,BW,44,71.47,5.64,0,-21.17,27.51
379,salalah,OM,37,71.60,4.70,0,17.02,54.09
389,christchurch,NZ,35,71.60,4.70,0,-43.53,172.63
392,hurghada,EG,27,77.00,9.17,0,27.26,33.81


In [9]:
#Hotel Map


In [10]:
#Store into variable named hotel_df.
hotel_df = pd.DataFrame(narrow_df, columns=["City", "Country","Humidity","Max Temp", "Wind Speed", "Cloudiness","Lat","Lng"])
#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Humidity,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Hotel Name
0,hobart,AU,54,60.01,9.17,75,-42.88,147.33,
1,anadyr,RU,100,23.00,4.47,75,64.75,177.48,
2,bluff,NZ,51,71.01,15.01,95,-46.60,168.33,
3,tiznit,MA,69,62.10,0.87,62,29.58,-9.50,
4,cape town,CL,44,60.80,11.41,2,-36.41,-71.90,
...,...,...,...,...,...,...,...,...,...
520,podosinovets,RU,97,37.65,7.25,100,60.28,47.07,
521,yar-sale,RU,95,20.97,8.72,90,66.83,70.83,
522,shubarkuduk,KZ,76,38.37,18.14,100,49.14,56.49,
523,sandpoint,US,92,33.01,5.82,90,48.28,-116.55,


In [12]:

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "location": "location",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
     
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['results'][0]['name']
        
    except:
        print("Missing fiels/Result..... Skipping")
     
    

Missing fiels/Result..... Skipping
Missing fiels/Result..... Skipping
Missing fiels/Result..... Skipping
Missing fiels/Result..... Skipping


In [14]:
hotel_df

,City,Country,Humidity,Max Temp,Wind Speed,Cloudiness,Lat,Lng,Hotel Name
0,hobart,AU,54,60.01,9.17,75,-42.88,147.33,Hobart
1,anadyr,RU,100,23.00,4.47,75,64.75,177.48,Anadyr
2,bluff,NZ,51,71.01,15.01,95,-46.60,168.33,Invercargill
3,tiznit,MA,69,62.10,0.87,62,29.58,-9.50,Ouijjane
4,cape town,CL,44,60.80,11.41,2,-36.41,-71.90,San Carlos
...,...,...,...,...,...,...,...,...,...
520,podosinovets,RU,97,37.65,7.25,100,60.28,47.07,Podosinovets
521,yar-sale,RU,95,20.97,8.72,90,66.83,70.83,Yar-Sale
522,shubarkuduk,KZ,76,38.37,18.14,100,49.14,56.49,Shubarkuduk
523,sandpoint,US,92,33.01,5.82,90,48.28,-116.55,Sandpoint


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))